<a href="https://colab.research.google.com/github/leepoweii/rag-homework/blob/main/%E9%85%92%E5%90%A7RAG%E5%93%81%E7%89%8C%E5%A4%A7%E4%BD%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用的資料說明

我在金門開了一間酒吧叫做夢酒館，正好最近在寫補助案，想到可以做一個 RAG 的內部資料集，因為常常在申請政府案子，很常需要重複整理有關公司資訊等等的，因此認為可以把相關資訊都全部放進 RAG，未來可以根據不同的專案，提取資料並且快速整合撰寫成不同方向的計劃書。

目前因為是作業實作，先將機器人設定為是對外的品牌介紹人員，分享給朋友體驗的時候會比較有趣。

### 目前裡面的資料有
1. 品牌介紹
2. 調酒酒單介紹
3. 相關媒體報導

### 發現比較大的問題如下:
* 複合式問句： 發現 RAG 系統一次只能處理單一問題，對於包含多個子句的複合式問句，無法精確理解並提供完整答案。
    * 目前解法：為了解決這個問題，利用 LLM 先將複合式問句拆解成多個子問題，分別進行檢索，最後再整合所有答案。


* 內容組合錯誤： 由於 RAG 系統將內容切割成 chunks，在檢索過程中，偶爾會出現內容組合錯誤，例如詢問 A 調酒卻得到 B 調酒的介紹。目前已發現問題，但仍在尋找最佳解法。
    * 目前解法：有發現問題但暫時還沒辦法完全理解和實作出解法，因此還沒有更新到程式內。



## 1. 下載資料與安裝並引入必要套件

In [ ]:
# prompt: 用 Linux 指令，讀入一個在網路上的，網址是URL的檔案

URL = "https://drive.google.com/uc?export=download&id=1uvu7kCV5z2l2AP1-enJulx-6xRwtCmf8"

!wget -O faiss_db.zip "$URL"
!unzip faiss_db.zip

!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

--2025-07-22 15:22:15--  https://drive.google.com/uc?export=download&id=1uvu7kCV5z2l2AP1-enJulx-6xRwtCmf8
Resolving drive.google.com (drive.google.com)... 142.251.179.100, 142.251.179.102, 142.251.179.139, ...
Connecting to drive.google.com (drive.google.com)|142.251.179.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1uvu7kCV5z2l2AP1-enJulx-6xRwtCmf8&export=download [following]
--2025-07-22 15:22:15--  https://drive.usercontent.google.com/download?id=1uvu7kCV5z2l2AP1-enJulx-6xRwtCmf8&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.179.132, 2607:f8b0:4004:c1f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.179.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49547 (48K) [application/octet-stream]
Saving to: ‘faiss_db.zip’

faiss_db.zip        100%[===================>]  48.39K  --.-KB/

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from openai import OpenAI
import gradio as gr
import ast

## 2. 自訂 E5 embedding 類別

In [ ]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

## 3. 載入 `faiss_db`

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

## 4. 設定好我們要的 LLM

In [ ]:
import os
from google.colab import userdata

api_key = userdata.get('OpenAI')
os.environ["OPENAI_API_KEY"] = api_key

model = "gpt-4.1-nano-2025-04-14"

client = OpenAI(
)

## 6. 使用 RAG 來回應

為了解決，複合式問句的檢索準確度，重新設計檢索和回答的流程


1. 語意拆解： 使用大型語言模型（LLM）將使用者輸入的複合式問題，拆解成多個語意清晰、可獨立檢索的子問題。
2. 個別檢索： 針對每個拆解後的子問題，分別從知識庫中檢索相關文件。
3. 整合回答： 根據原始問句，以及步驟二中檢索到的所有相關文件，使用 LLM 整合生成最終的完整回覆。

### 6.1 語意拆解
透過 LLM 將 user input 拆解成很多個小子句


In [ ]:
# 透過語意拆解對話成很多個小子句

system_prompt_seperate = """
你是一位專門協助語意拆解的 AI 工具設計師，任務是將使用者輸入的複合問題，轉換成清楚、可獨立檢索的子問題。

請依照以下規則進行拆解：

1. 若問題中包含兩個以上的子句（例如多個動作、主詞、或時間條件），請將其拆成多個子問題。
2. 每個子問題應該是完整的句子，能夠獨立被語意檢索模型理解。
3. 拆解後請用 Python list 格式回傳，例如：[夢酒館登過哪些國際媒體，分別是哪些調酒？]，請拆解成
["夢酒館有登過哪些國際媒體？", "夢酒館登上國際媒體的是哪些調酒？"]
4. 若問題本身已經是單一子句，請回傳原句構成的 list。
5. 除了 Python list 之外，不要回傳任何說明。

請注意：不要自行補充或改寫原始問題的語意，只做語意拆解。
"""

input_prompt_seperate = """
請將下列文字，重新拆解成不同的子句，並且整理成一個 Python list 回傳。
{queries}
"""

def seperate_queries(user_input):
    # 將自定 prompt 套入格式
    final_prompt = input_prompt_seperate.format(queries=user_input)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_seperate},
            {"role": "user", "content": final_prompt},
        ]
    )
    raw_output = response.choices[0].message.content
    sub_queries = ast.literal_eval(raw_output)

    return {
        "main_query": user_input, # 把原始問題也保留
        "sub_queries": sub_queries
    }

# 測試與示範
user_input = "我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。"

# 將主要輸入分解成多個子句
result = seperate_queries(user_input)
main_query = result['main_query']
queries_list = result['sub_queries']

# 輸出原始問句和分解後的子句
print(f"原始問句是：\n{result['main_query']}\n")
print("分解後的子問句是：")
for query in queries_list:
    i = queries_list.index(query) + 1
    print(f"子問句{i}：{query}")

原始問句是：
我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。

分解後的子問句是：
子問句1：我喜歡清爽氣泡的調酒。
子問句2：請你推薦我一杯調酒。
子問句3：請你跟我說明夢酒館在地方所做的貢獻。
子問句4：請你說明夢酒館登過的國際媒體報導。


### 6.2 個別檢索
分別針對每一個子句檢索資料

In [ ]:
#分別檢索取回資料並使用 top k = 3 讓資料更完整
def retrieve_answers(queries_list, top_k=3):
    results = []
    for sub_query in queries_list:
        docs = retriever.invoke(sub_query, config=dict(k=top_k)) # Gemini 說 BaseRetriever.get_relevant_documents 之後不支援，要我改成 invoke
        # 做一個空白的 list 並存入檢索後的資料
        retrieved_chunks = []

        for doc in docs:
            retrieved_chunks.append(doc.page_content)
        results.append({
            "sub_query": sub_query,
            "chunks": retrieved_chunks
        })
    return results

# 測試與示範(延續上面的 user_input)
retrieved_answers = retrieve_answers(queries_list)

# 輸出每一個子句檢索回來的答案
print(f"共有{len(retrieved_answers)}組問答，Q&A分別是:\n")

for answer_pairs in retrieved_answers:
    sub_query = answer_pairs["sub_query"]
    chunks = answer_pairs["chunks"]
    print(f"Sub_query: {sub_query}\n Chunks: {chunks}\n")

共有4組問答，Q&A分別是:

Sub_query: 我喜歡清爽氣泡的調酒。
 Chunks: ['**原料**  \n琴酒（琴酒）、百香果、芳香萬壽菊、茉莉綠茶、蜂蜜、氣泡水\n\n**原料意象**  \n蜂蜜綠茶：清新、甜美的湖邊約會。芳香萬壽菊：香草類植物，湖畔草地的感覺。氣泡調酒：輕盈爽口，輕鬆日常的約會感。\n\n**製作技法**  \n奶洗：原理是牛奶加入酸性物質會凝固，咖啡濾紙過濾掉凝固的東西後就可以得到澄清的液體。我們的做法是，把泡得過濃的蜂蜜綠茶（要澄清但又要茶味夠重所以我們刻意泡超久）還有其他外料都加好，接著加入牛奶，等他凝固後，最後全部過濾完，奶洗的過程剛好會把單寧一起濾掉，得到有茶味但是不苦澀的蜂蜜綠茶。\n\n**風味描述**  \n清爽、淡雅、氣泡感明顯，有如蜂蜜綠茶般的親切甜感，帶有花香與果酸，入口輕盈、毫無負擔。宛如初戀般的清新甜蜜，適合任何時刻來上一杯。\n\n**適合對象**  \n喜歡清爽、無負擔調酒的人；喜愛花果茶系風味、偏好輕酒感者；尋找一杯日常也能喝的低壓力調酒者。\n\n**視覺意象**  \n金黃蜂蜜色、自然、愜意、午後陽光下的草皮約會感。\n\n**隨杯明信片內容內容**  \n天鵝船\n\n\n---', '**適合對象**  \n喜歡清爽、無負擔調酒的人；喜愛花果茶系風味、偏好輕酒感者；尋找一杯日常也能喝的低壓力調酒者。\n\n**視覺意象**  \n金黃蜂蜜色、自然、愜意、午後陽光下的草皮約會感。\n\n**隨杯明信片內容內容**  \n天鵝船\n\n\n---\n\n---\n調酒名稱: "小鳳出逃"\n價格: 360\n酒精濃度: 9.48\ntags: [\'金門調酒\', \'特色酒單\']\n---\n\n**故事**  \n1999年的颱風把畜試所的孔雀園吹爆造成孔雀跑出去繁殖。颱風把畜試所的孔雀園吹爆造成孔雀跑出去繁殖，原先設計時以 PAVAN 藍孔雀葡萄利口酒，和金門野外也都會看到的迷迭香，來表示現在金門野外都會看到的藍孔雀。希臘琴酒，洋乳香（一個特殊的樹脂）特殊氣味和其他材料帶來的木質調性，把迷迭香和葡萄風味連接在一起。（現在因為 PAVAN 台灣斷貨，所以換成台灣產的麝香葡萄利口酒）\n\n**原料**  \n琴酒、麝香葡萄利口酒、迷迭香、氣泡水\n\n**原料意象**  \n氣泡水：象徵颱

### 6.3 整合回答
把6.1 6.2得到的資料，請 LLM 整合並回答。

In [ ]:
#使用 user input　當成問句，並使用檢索之後的資料提共給　LLM 當作內容基準，可以避免分割成子句後，失去了原本完整問句的語意和情緒，但又可以透過檢索回來的資料，提升回答準確度。

system_prompt = (
    "你是『把夢酒館介紹給旅客的品牌大使』—— 一位兼具雞尾酒專業、地方文化策展力的品牌介紹人員。"
    "你的任務是依據被提共的內容，親切生動地向來訪旅客講解夢酒館的故事、雞尾酒靈感及我們在金門推動的地方創生行動。"
    "請以繁體中文作為回答語言，回答要溫暖、專業。"
    "請根據來源資料回答，若資料中未明確提及，請不要自行組合資訊。"
)

prompt_template = """
旅客問題：
{main_query}

根據下列資料來回應訪客問題，務必貼合資料細節並保持品牌語氣：
{retrieved_answers}

請依據下列規範作答：
1. 可以的話，附上相關連結或實際案例說明。
2. 如果知識庫無法支援完整答案，請誠實說明並提出可協助的後續方向 (ex. 請 Email 到 contact@mojokm.com)，可以推薦一個最接近的，但不要自行組合資訊。
3. 請一定要全程使用台灣人慣用的繁體中文來回答。
4. 在推薦調酒之前，請先了解對方的口味喜好，否則以品牌故事理念為主要回答。
5. 請避免將不同調酒的描述混合使用，除非資料中有明確比較。
"""

chat_history = []

def integrate_answers(main_query, retrieved_answers):

    # 將自定 prompt 套入格式
    final_prompt = prompt_template.format(main_query=main_query, retrieved_answers=retrieved_answers)

    # 呼叫 OpenAI API
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": final_prompt},
    ]
    )
    answer = response.choices[0].message.content

    chat_history.append((main_query, answer))
    return answer

# 測試與示範
print(f"原始問句是:\n\n「{main_query}」\n\n將會使用這句話當作主要的問句給LLM\n\n\n")

# 整合檢索之後的回答
answer = integrate_answers(main_query, retrieved_answers)
print(f"整合之後的回答是:\n\n{answer}")
print('\n\n\n')

原始問句是:

「我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。」

將會使用這句話當作主要的問句給LLM



整合之後的回答是:

您好，非常感謝您的喜愛與詢問！由於您提到喜歡清爽氣泡的調酒，我很推薦我們夢酒館的招牌調酒之一：「蜂蜜綠茶氣泡」來滿足您的口感喜好。

這款調酒靈感來自我們對於自然和日常的詮釋，採用琴酒、百香果、芳香萬壽菊、茉莉綠茶、蜂蜜，再加入氣泡水，喝起來清新、淡雅，帶有花香與果酸，入口輕盈毫無負擔，宛如初戀般的純粹甜蜜，非常適合喜愛輕盈氣泡感的朋友們！視覺上則是金黃蜂蜜色，營造午後陽光下的草皮約會情境。這款調酒除了美味外，也象徵我們對自然與日常生活的熱愛與追求，在金門推動地方創生和文化交流方面，我們積極與在地團隊合作，透過舉辦活動、合作地方大學、連結青創品牌，串聯社群，讓在地文化得以活化與傳承。同時，我們的故事和創新也受到國際媒體認可，像是AsiaOne、Straits Times、AOL、SCMP等平台都曾轉載我們的相關報導，特別是我們與Reuters合作的歷史靈感調酒"Pick and Eat"，在國際上也獲得了不少關注。

如果您想聽聽更詳細的故事或有特別喜好的口味，也歡迎跟我分享，我可以再為您做更個人化的推薦喔！期待您來夢酒館，一起品味這份屬於金門的風味與文化。






# 7. 用 Gradio 打造 Web App

In [ ]:
# 把上面三個步驟重新整合成一個 function 給 Gradio 使用

chat_history_local = []

def respond(message, chat_history_local):
    result = seperate_queries(message)
    main_query = result['main_query']
    queries_list = result['sub_queries']
    retrieved_answers = retrieve_answers(queries_list)
    response = integrate_answers(main_query, retrieved_answers)
    # 跳出警告說新版要指定其他 type 而不是用預設 tuple 來儲存，參考 Gemini 說明修改成這個格式
    chat_history_local.append({"role": "user", "content": message})  # 使用者訊息
    chat_history_local.append({"role": "assistant", "content": response})  # 機器人回應
    return "", chat_history_local

#測試與示範
res = respond(user_input, chat_history_local)
res

('',
 [{'role': 'user',
   'content': '我喜歡清爽氣泡的調酒，請你推薦我一杯調酒。同時也請你跟我說明夢酒館在地方所做的貢獻，還有夢酒館登過的國際媒體報導。'},
  {'role': 'assistant',
   'content': '您好，非常感謝您對夢酒館的興趣！如果您喜愛清爽氣泡的調酒，我們特別推薦「蜂蜜綠茶氣泡」這款，它融合了茉莉綠茶、百香果、芳香萬壽菊與氣泡水，帶來清新、淡雅又不失花果香的風味，入口輕盈，像是一場湖邊午後的悠閒約會。它的金黃蜂蜜色還讓人聯想到午後陽光下的草皮情境，非常適合喜歡輕鬆無負擔的朋友。如果您想了解更細緻的描述，也可以詢問，我很樂意為您介紹。\n\n除了飲品的部分，夢酒館一直在地方的文化創生方面付出不少心力。我們不僅在品牌空間與調酒故事中融入金門在地元素，還積極與在地社區合作，如與金門大學推出聯名調酒、舉辦客座調酒師交流活動，以及與青年團隊共同策劃文化活動，促進地方文化傳承與青年創業精神。此外，我們還透過一系列的地方實踐，打造商圈與社群的連結，帶動在地創生的能量。\n\n關於國際媒體的曝光方面，夢酒館的故事曾經登上多個國際平台。其中最具代表性的媒體之一是Reuters，它於2024年5月的報導中介紹我們的代表作「Pick and Eat」調酒，融合了金門戰地歷史與創新元素，彰顯台灣酒吧將文化與味覺結合的力量。這篇報導不僅被Reuters轉載，也同步被AsiaOne、Straits Times、AOL及SCMP等媒體再度報導，展現我們在國際舞台上的品牌影響力與故事傳播力。\n\n如果您想更詳細瞭解我們的故事，或是其他特色調酒，也可以隨時與我聯繫，或者參考官方網站與媒體連結。再次感謝您的支持，希望您在夢酒館能找到屬於自己的一杯清新好茶！'}])

In [13]:
# 清空對話紀錄
chat_history = []
chat_history_local = []

with gr.Blocks() as demo:
    gr.Markdown("""
    # 夢酒館品牌大使 \n
    您好，我是夢酒館的品牌大使，可以問我任何有關夢酒館的問題，請問有什麼我可以為您服務的嗎？\n
    可以問我有關\n
    1. 調酒推薦 2. 我們的故事 3. 其他媒體報導
    """)

    chatbot = gr.Chatbot(type="messages") # 根據 Gemini 的說明，修改成新版支援的格式。

    msg = gr.Textbox(placeholder="請輸入你對夢酒館的好奇...")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5e10ba9574173a2d96.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5e10ba9574173a2d96.gradio.live


In [12]:
# 聊完天後，輸出聊天紀錄
index = 0
for chat in chat_history:
    index +=1
    print(f"{index}:\n使用者輸入:{chat[0]}\nRAG機器人:{chat[1]}\n\n")